In [ ]:
import json
import requests
import datetime
from os import getenv
from base64 import b64encode
from datetime import datetime
from dataclasses import dataclass, field
from typing import List
from dotenv import load_dotenv

load_dotenv()

In [ ]:
usuario = getenv("USR_ALEGRA")
token = getenv("TKN_ALEGRA")
url_facturas = "https://api.alegra.com/api/v1/invoices"
url_clientes = "https://api.alegra.com/api/v1/contacts"
url_consulta = "https://api.alegra.com/api/v1/contacts/id?url=query%3D223423423"
headers = {
    "Authorization": f'Basic {b64encode(f"{usuario}:{token}".encode("utf-8")).decode("UTF-8")}',
    "Accept": "application/json",
    "Content-Type": "application/json"
}
payload_factura = {
    "items": [
        {
            "tax": [{"id": 3}],
            "id": 1,
            "name": "Nombre del Producto Primero",
            "discount": 0,
            "price": 42017,
            "quantity": 1
        },
        {
            "tax": [{"id": 3}],
            "id": 1,
            "name": "Nombre del Producto Segundo",
            "discount": 10,
            "price": 42017,
            "quantity": 1
        },
        {
            "tax": [{"id": 3}],
            "id": 1,
            "name": "Nombre del Producto Tercero",
            "discount": 20,
            "price": 42017,
            "quantity": 1
        }
    ],
    "payments": [
        {
            "account": {"id": 4},
            "retentions": [
                {
                    "id": 6,
                    "amount": 1000
                },
                {
                    "id": 11,
                    "amount": 1000
                }
            ],
            "date": "2022-12-21",
            "paymentMethod": "cash",
            "amount": 43000
        }
    ],
    "client": {"id": 2},
    "stamp": {"generateStamp": "true"},
    "paymentMethod": "CASH",
    "paymentForm": "CASH",
    "type": "NATIONAL",
    "operationType": "SALE",
    "numberPurchaseOrder": "OrdenServicio000",
    "status": "open",
    "saleType": "SERVICE",
    "dueDate": "2022-12-21",
    "date": "2022-12-21"
}
payload_cliente = {
    "address": {
        "city": "Acacías",
        "department": "Meta",
        "country": "Colombia",
        "address": "plaza lucena"
    },
    "nameObject": {
        "firstName": "Nombre",
        "lastName": "de Persona"
    },
    "identificationObject": {
        "number": "1231232288",
        "dv": 9,
        "type": "NIT"
    },
    "name": "Nombre de empresa",
    "kindOfPerson": "LEGAL_ENTITY",
    "regime": "COMMON_REGIME",
    "phonePrimary": "1111111111",
    "mobile": "444444444",
    "email": "pruebacol@gmail.com",
    "ignoreRepeated": False,
    "type": "client",
}

In [ ]:
class DriverAlegra:
    """Clase con los métodos directos para establecer conexión con el servicio de Alegra, y gestionar
    los posibles mensajes que se reciben desde allá.
    """

    def obtener_autenticacion_base64(self) -> str:
        """Método de clase que invoca las constantes de la configuración, y genera la cadena con el token
        codificado, para la autenticación frente al servicio de Alegra.

        Returns:
            str: Token codificado en base64, y convertido a cadena de texto
        """
        return b64encode(f"{usuario}:{token}".encode("utf-8"))

    def enviar_request(
        self, url: str, payload: dict = {}, parametros: dict = {}, metodo: str = "GET"
    ) -> dict:

        headers = {
            "Accept": "application/json",
            "Authorization": f"Basic {self.obtener_autenticacion_base64().decode('utf-8')}",
            "Content-Type": "application/json",
        }
        try:
            if metodo == "POST":
                response = requests.post(
                    url,
                    json=payload,
                    headers=headers,
                )
            elif metodo == "GET":
                response = requests.get(
                    url,
                    params=parametros,
                    json=payload,
                    headers=headers,
                )

            response.raise_for_status()

        except requests.exceptions.HTTPError as e:
            return {
                "estado": "error",
                "codigo": response.status_code,
                "categoria": "Error al enviar información al servicio de Alegra",
                "detalle": response.json().get("message", "Error indeterminado"),
                "url": url,
                "respuesta_servicio": f"{response.json()} - {e}",
            }            

        except requests.exceptions.RequestException as e:
            return {
                "estado": "error",
                "codigo": response.status_code,
                "categoria": "Error indeterminado",
                "detalle_error": str(e),
                "url": url,
                "respuesta_servicio": response.json(),
            }            

        return response.json()

In [ ]:
consulta = DriverAlegra()
resp = consulta.enviar_request(url_clientes, {}, {"query": 223423423})

In [ ]:
def _calcular_dv(numero_documento):
        factores = [3, 7, 13, 17, 19, 23, 29, 37, 41, 43, 47, 53, 59, 67, 71]
        rut_ajustado = str(numero_documento).rjust(15, "0")
        sumatoria = sum(int(rut_ajustado[14 - i]) * factores[i] for i in range(14)) % 11
        if sumatoria > 1:
            return 11 - sumatoria

        else:
            return sumatoria

In [ ]:
_calcular_dv(899999239)

In [ ]:
"""
Valida si el número y el dígito de verificación corresponden
>>> validar(811026552,0)
False
>>> validar(890925108,6)
True
>>> validar(800197384,1)
False
>>> validar(899999034,1)
True
>>> validar(8600123361,3)
False
>>> validar(899999239,2)
True
>>> validar(890900841,7)
False
"""
9.66/1000

In [ ]:
@dataclass
class ProductoAlegra:
    identificador: int
    nombre: str
    precio: int
    cantidad: int = 1
    descuento: int = 0
    impuesto: list = (3,)

    def calcular_monto_pago(self) -> float:
        return (self.precio - (self.precio * self.descuento / 100)) * 1.19

    def serializar_producto(self):
        return {
            "tax": [{"id": self.impuesto[0]}],
            "id": self.identificador,
            "name": self.nombre,
            "discount": self.descuento,
            "price": self.precio,
            "quantity": self.cantidad,
        }

@dataclass
class PagoAlegra:
    monto_pago: int
    cuenta_bancaria: int
    metodo_pago: str = "cash"
    reteica: tuple = (False, 4, 0)
    retefuente: tuple = (False, 11, 0)
    fecha_pago: str = datetime.now().strftime("%F")
    
    def serializar_pago(self):
        return {
            "account": {"id": self.cuenta_bancaria},
            "retentions": [
                {"id": self.reteica[1], "amount": self.reteica[2]} if self.reteica[0] is True else None,
                {"id": self.retefuente[1], "amount": self.retefuente[2]} if self.retefuente[0] is True else None,
            ] if any((self.reteica[0], self.retefuente[0])) else [],
            "date": self.fecha_pago,
            "paymentMethod": "cash",
            "amount": self.monto_pago
        }

@dataclass
class ListaProductosAlegra:
    productos: list[ProductoAlegra]

    def sumar_valor_productos(self):
        return sum([producto.calcular_monto_pago() for producto in self.productos])

    def serializar_productos(self) -> list:
        return [producto.serializar_producto() for producto in self.productos]

@dataclass
class ListaPagosAlegra:
    pagos: list[PagoAlegra]
    
    def serializar_pagos(self):        
        return [pago.serializar_pago() for pago in self.pagos]

In [ ]:
producto_uno = ProductoAlegra(
    1,
    "Producto uno",
    10000,
    descuento=20
)
producto_dos = ProductoAlegra(
    1,
    "Producto dos",
    20000,
    descuento=10
)
producto_tres = ProductoAlegra(
    1,
    "Producto tres",
    30000,
    descuento=5
)
producto_cuatro = ProductoAlegra(
    1,
    "Producto tres",
    30000,
    descuento=0
)
pago_uno = PagoAlegra(
    50000,
    4
)
pago_dos = PagoAlegra(
    50000,
    4,
    reteica = (True, 4, 1000)
)
pago_tres = PagoAlegra(
    50000,
    4,
    retefuente = (True, 11, 1000)
)
pago_cuatro = PagoAlegra(
    50000,
    4,
    reteica = (True, 4, 1000),
    retefuente = (True, 11, 1000)
)

In [ ]:
producto_uno.calcular_monto_pago()

In [ ]:
lista_productos = ListaProductosAlegra([producto_uno, producto_dos, producto_tres, producto_cuatro])
lista_pagos = ListaPagosAlegra([pago_uno, pago_dos, pago_tres, pago_cuatro])

In [ ]:
lista_productos.sumar_valor_productos()

In [ ]:
lista_productos.serializar_productos()

In [ ]:
pago_tres.serializar_pago()

In [ ]:
lista_pagos.serializar_pagos()